# Training and Evaluation

We will take a first-pass at evaluating or technique to start understanding its efficacy. We will existing CNN architectures and evaluate its performance on our interested categories with and without using our interested categories.

In [1]:
import cv2
import datetime
from matplotlib import pyplot as plt
import numpy as np
import sys
import tensorflow as tf
print('TensorFlow Version: ', tf.__version__)

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

np.random.seed(123)

TensorFlow Version:  2.1.0


In [24]:
# Set hyperparameters for training & validation
INPUT_SHAPE = (64, 64, 3)
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.10
TRAIN_STEPS_PER_EPOCH = 1200
TEST_STEPS = 500
NUM_EPOCHS = 250

In [3]:
# Define utilities and helper functions
# NOTE: Copied from clustering NB
def load_metadata(filename):
    with open(filename, 'r') as f:
        return [x.strip().split('\t') for x in f.readlines()]
    
@tf.function
def decode_img(image):
    img = tf.image.decode_jpeg(image, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [64, 64])

@tf.function
def load_image_data(path, label):
    img_data = tf.io.read_file(path)
    img = decode_img(img_data)
    return img, label

def load_labels(metadata):
    labels = np.array([x[1] for x in metadata])
    distinct_labels = np.array([[x] for x in set(labels)])
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(distinct_labels)
    y_train = encoder.transform([[x] for x in labels])
    return (y_train, encoder)

In [35]:
# Create functions for three models: (i) custom, simple CNN, (ii) MobileNet + FCs, and (iii) VGG16 + FCs
def get_simplecnn(input_shape=INPUT_SHAPE):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(512, (3, 3), (1, 1), input_shape=input_shape, activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(512, (2, 2), (1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (2, 2), (1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (2, 2), (1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(200, activation='softmax'),
    ])

def get_mobilenet(input_shape=INPUT_SHAPE):
    application = tf.keras.applications.MobileNet(input_shape=input_shape, include_top=False)
    for i in range(len(application.layers)):
        application.layers[i].trainable = False
        
    return tf.keras.Sequential([
        application,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(200, activation='softmax')
    ])

def get_vgg16(input_shape=INPUT_SHAPE):
    application = tf.keras.applications.VGG16(input_shape=input_shape, include_top=False)
    for i in range(len(application.layers)):
        application.layers[i].trainable = False
        
    return tf.keras.Sequential([
        application,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(200, activation='softmax')
    ])

In [36]:
simplecnn = get_simplecnn()
simplecnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 62, 62, 512)       14336     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 31, 512)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 30, 512)       1049088   
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 512)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 256)       524544    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 6, 256)        

In [37]:
mobilenet = get_mobilenet()
mobilenet.summary()

/Users/uzaymacar/anaconda3/envs/newTF/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 2, 2, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 200)               205000    
Total params: 3,433,864
Trainable params: 205,000
Non-trainable params: 3,228,864
_________________________________________________________________


In [38]:
vgg16 = get_vgg16()
vgg16.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 200)               205000    
Total params: 17,017,864
Trainable params: 2,303,176
Non-trainable params: 14,714,688
_________________________________________________________________


In [39]:
# TODO: Remove hardcoding
print('Loading data into memory...')
train_metadata = load_metadata('./metadata_output/train_metadata.txt')
(y_train, encoder) = load_labels(train_metadata)

# Interested indices for test data filtering
interested_categories = ['n01882714', 'n04562935']
interested_one_hot = encoder.transform([[x] for x in interested_categories])
interested_indices = np.array([x[1] for x in np.argwhere(interested_one_hot == 1)])
print('Done.')

Loading data into memory...
Done.


In [40]:
# Encoding sanity checks;
# assert(len(train_metadata) == len(y_train))
# assert(len(set(y_train)) == 200)
assert(np.count_nonzero(y_train == 1) == len(train_metadata))
# print(y_train)

## (1) BASELINE MODEL: VGG16

In [41]:
# Get all data
paths_and_labels = [(train_metadata[x][0], y_train[x]) for x in range(len(y_train))]
print('Num. Total Images: ', len(paths_and_labels))

# Split data into train and validation sets
np.random.shuffle(paths_and_labels)
num_validation = int(len(paths_and_labels) * VALIDATION_SPLIT)
train_paths_and_labels = paths_and_labels[num_validation:]
validation_paths_and_labels = paths_and_labels[:num_validation]
print('Num. Train Images: ', len(train_paths_and_labels))
print('Num. Validation Images: ', len(validation_paths_and_labels))

# Convert training set into a TF dataset via generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_paths_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train[0])]))
)
train_dataset = train_dataset.map(lambda x,y: load_image_data(x, y), 
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset = train_dataset.cache()
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Convert validation set into a TF dataset via generator
validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_path_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train[0])]))
)
validation_dataset = validation_dataset.map(lambda x,y: load_image_data(x, y), 
                                            num_parallel_calls=tf.data.experimental.AUTOTUNE)

validation_dataset = validation_dataset.cache()
validation_dataset = validation_dataset.repeat()
validation_dataset = validation_dataset.batch(1)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Num. Total Images:  70000
Num. Train Images:  63000
Num. Validation Images:  7000


In [45]:
def train_model(model, train_dataset, validation_dataset, name):    
    # Compile model                                                                                                      
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=4e-4),                                                           
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),                                  
                  metrics=['accuracy'])      
    
    # Stop early if we're not making good progress                                                                           
    early_stop_monitor = tf.keras.callbacks.EarlyStopping(monitor='val_loss',                                                                                              
                                                          restore_best_weights=True,                                                                                       
                                                          patience=10)   

    # Prepare for checkpoints            
    checkpoint_path = './checkpoints/' + name + '/cp-{epoch:04d}.ckpt'                                   
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,                                                                                    
                                                     verbose=1,                                                                                                   
                                                     save_weights_only=False,                                                                                     
                                                     save_freq=25000000)

    # Tensorboard                                                                                                        
    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")                                              
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    history = model.fit(x=train_dataset,
                        epochs=NUM_EPOCHS,                                                                                                  
                        steps_per_epoch=TRAIN_STEPS_PER_EPOCH,
                        callbacks=[tensorboard_callback, cp_callback, early_stop_monitor],
                        use_multiprocessing=True,
                        validation_steps=num_validation,
                        validation_data=validation_dataset,
                        shuffle=True)

    return history

In [46]:
# Evaluate model on interesting inputs
def evaluate_model(model, test_sets):
    for test_set in test_sets:
        X = test_set[0]
        y = test_set[1]
        model.evaluate(X, y)

In [ ]:
# Train and save model
train_model(mobilenet, train_dataset, validation_dataset, 'mobilenet_imbalanced')

Train for 1200 steps, validate for 7000 steps
Epoch 1/250
  27/1200 [..............................] - ETA: 22:10 - loss: 5.2965 - accuracy: 0.0069

In [ ]:
if not os.path.exists(os.path.join('models', 'mobilenet_imbalanced')):
    os.makedirs(os.path.join('models', 'mobilenet_imbalanced'))
    
mobilenet.save(os.path.join('models', 'mobilenet_imbalanced'))
print('model saved')

In [ ]:
# TODO: Load test data, filter for interest, evaluate model

## (2) MOBILENET + STANDARD AUGMENTATIONS

In [ ]:
# TODO: We have to somehow incorporate the below with tf.Datasets
train_datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.4,
    height_shift_range=0.4,
    zoom_range=[0.4, 1.6],
    horizontal_flip=True,
    brightness_range=(0.6, 1.4),
    fill_mode='nearest',
)

## (3) MOBILENET + SINGAN